In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
dev = qml.device("lightning.qubit", wires=1)
@qml.qnode(dev, interface="autograd", diff_method="parameter-shift")
def circuit(params):
    qml.RY(params, wires=0)
    qml.RY(0, wires=0)
    return qml.expval(qml.PauliZ(0))
params = np.array([0.1], requires_grad=True)
# qml.jacobian(circuit)(params)

In [6]:
class NeuralNetwork:
    def __init__(self, layers, alpha=0.1):
        self.W = []
        self.layers = layers
        self.alpha = alpha

        for i in np.arange(0, len(layers) - 2):
            
            w = np.random.randn(layers[i] + 1, layers[i + 1] + 1)
            self.W.append(w / np.sqrt(layers[i]))

        w = np.random.randn(layers[-2] + 1, layers[-1])
        self.W.append(w / np.sqrt(layers[-2]))

    def __repr__(self):
        return "NeuralNetwork: {}".format(
			"-".join(str(l) for l in self.layers))

    def sigmoid(self, x):
        out = []
        for i in x[0]:
            ans = circuit(i*np.pi)
            out.append(ans)
        return np.array([out])

    def sigmoid_deriv(self, x):
        out = []
        for i in x[0]:
            ans = qml.jacobian(circuit)(i*np.pi)
            out.append(ans)
        return np.array([out])

    def fit(self, X, y, epochs=1000, displayUpdate=100):
        X = np.c_[X, np.ones((X.shape[0]))]
        for epoch in np.arange(0, epochs):
            for (x, target) in zip(X, y):
              self.fit_partial(x, target, epoch)
              
    def fit_partial(self, x, y, test):
        
        A = [np.atleast_2d(x)]
        
        for layer in np.arange(0, len(self.W)):
            net = A[layer].dot(self.W[layer])
            
            out = self.sigmoid(net)
            # print(out, 'h')
            
            A.append(out)
            
        error = A[-1] - y
        # print(self.sigmoid_deriv(A))
        D = [error * self.sigmoid_deriv(A[-1])]

        for layer in np.arange(len(A) - 2, 0, -1):
            
            delta = D[-1].dot(self.W[layer].T)
            delta = delta * self.sigmoid_deriv(A[layer])
            D.append(delta)

        D = D[::-1]
        
        for layer in np.arange(0, len(self.W)):

            self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])

    def predict(self, X, addBias=True):
        
        p = np.atleast_2d(X)

        if addBias:
            p = np.c_[p, np.ones((p.shape[0]))]
            
        for layer in np.arange(0, len(self.W)):
            p = self.sigmoid(np.dot(p, self.W[layer]))

        return p

    def calculate_loss(self, X, targets):
        
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, addBias=False)
        loss = 0.5 * np.sum((predictions - targets) ** 2)

        return loss

print('hi')

hi


In [7]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

In [13]:
nn = NeuralNetwork([2, 2, 1], alpha=0.5)
nn.fit(X, y, epochs=10000, displayUpdate=1000)

In [16]:
for (x, target) in zip(X, y):
	pred = nn.predict(x)[0][0]
	step = 1 if abs(pred) < 0.5 else 0
	print("[INFO] data={}, ground-truth={}, pred={:.4f}, step={}".format(
		x, target[0], pred, step))

[INFO] data=[0 0], ground-truth=0, pred=0.6074, step=0
[INFO] data=[0 1], ground-truth=1, pred=0.4130, step=1
[INFO] data=[1 0], ground-truth=1, pred=0.1005, step=1
[INFO] data=[1 1], ground-truth=0, pred=-0.9886, step=0
